## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [3]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

sys.path

['c:\\Users\\Nicolas R\\Documents\\random_sast\\ExperimentationRSAST',
 'c:\\ProgramData\\Anaconda3\\envs\\sast_env\\python38.zip',
 'c:\\ProgramData\\Anaconda3\\envs\\sast_env\\DLLs',
 'c:\\ProgramData\\Anaconda3\\envs\\sast_env\\lib',
 'c:\\ProgramData\\Anaconda3\\envs\\sast_env',
 '',
 'c:\\ProgramData\\Anaconda3\\envs\\sast_env\\lib\\site-packages',
 'c:\\ProgramData\\Anaconda3\\envs\\sast_env\\lib\\site-packages\\win32',
 'c:\\ProgramData\\Anaconda3\\envs\\sast_env\\lib\\site-packages\\win32\\lib',
 'c:\\ProgramData\\Anaconda3\\envs\\sast_env\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Nicolas R/random_sast/sast',
 'C:\\Users\\Nicolas R/random_sast/cd_diagram',
 'C:\\Users\\Nicolas R/random_sast/sast',
 'C:\\Users\\Nicolas R/random_sast/cd_diagram']

In [4]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

ModuleNotFoundError: No module named 'sast'

### Reading Datasets

In [ ]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [ ]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [ ]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.919203,0.181923,0.013529,0.127419,0.037550,0.002808,SonyAIBORobotSurface2,ACF&PACF: n_random_points=1 nb_inst_per_class=...,1,(max instances per class)//2,Rsast,ACF&PACF
1,1,0.828961,0.168610,0.021266,0.037572,0.106363,0.002882,SonyAIBORobotSurface2,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,1,Rsast,ACF&PACF
2,2,0.876180,1.377738,0.011656,0.138769,1.211099,0.015296,SonyAIBORobotSurface2,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,(max instances per class)//2,Rsast,ACF&PACF


In [ ]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [ ]:
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineAgeGroup', 'GunPoint',
       'ItalyPowerDemand', 'SmoothSubspace', 'SonyAIBORobotSurface2'],
      dtype=object)

In [ ]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")

df_result['dataset_name'].unique()


array(['SonyAIBORobotSurface2', 'GunPoint', 'Chinatown', 'SmoothSubspace',
       'Coffee', 'DistalPhalanxOutlineAgeGroup', 'ItalyPowerDemand'],
      dtype=object)

In [ ]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [ ]:

filter_param=df_result[df_result.rpoint=='(lenthg ts)//2']
filter_param.rpoint.unique()

array(['(lenthg ts)//2'], dtype=object)

In [ ]:

filter_param=filter_param[filter_param.nb_per_class=='(max instances per class)//2']
filter_param.nb_per_class.unique()

array(['(max instances per class)//2'], dtype=object)

In [ ]:
# create a pivot table with the mean of score by dataset

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,score,method
0,Chinatown,0.945578,0.971817,0.958017,0.971623,0.966375,0.974927,0.982507,ROCKET
1,Coffee,0.964286,1.000000,0.978571,0.935714,1.000000,1.000000,1.000000,ROCKET
2,DistalPhalanxOutlineAgeGroup,0.736691,0.723741,0.771942,0.725180,0.729496,0.720863,0.748201,ROCKET
3,GunPoint,0.830222,0.976000,0.845778,0.972444,0.965778,0.974667,1.000000,ROCKET
4,ItalyPowerDemand,0.949660,0.946615,0.952705,0.944866,0.951668,0.944866,0.968902,ROCKET
5,SmoothSubspace,0.870222,0.910667,0.879556,0.896889,0.876000,0.908889,0.980000,ROCKET
6,SonyAIBORobotSurface2,0.751242,0.899685,0.756348,0.866387,0.837356,0.893319,0.916055,ROCKET


In [ ]:
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='var')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
len_method_vs_ds

len_method,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF
0,Chinatown,0.000856,0.000100,0.000323,0.000162,0.000167,0.000062
1,Coffee,0.000000,0.000000,0.000383,0.000255,0.000000,0.000000
2,DistalPhalanxOutlineAgeGroup,0.000439,0.000175,0.000277,0.000193,0.000382,0.000170
3,GunPoint,0.000526,0.000088,0.000234,0.000050,0.000145,0.000097
4,ItalyPowerDemand,0.000109,0.000015,0.000055,0.000051,0.000029,0.000038
5,SmoothSubspace,0.001110,0.000316,0.000187,0.000266,0.001008,0.000341
6,SonyAIBORobotSurface2,0.004930,0.000157,0.003355,0.000337,0.001183,0.000283


In [ ]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.859593,0.787876,0.845655,0.867829,0.863819,0.861348
ACF&PACF,0.917369,0.888078,0.912854,0.917222,0.916539,0.915966
Max ACF,0.869080,0.799489,0.845175,0.876046,0.860507,0.873798
Max PACF,0.905625,0.860852,0.902515,0.897832,0.900279,0.899226
None,0.897099,0.856152,0.895193,0.896225,0.893050,0.898103
PACF,0.916214,0.891311,0.912397,0.915178,0.915779,0.915661


In [ ]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.856108,0.766801,0.848993,0.875039,0.868165,0.872629
ACF&PACF,0.919537,0.871071,0.909504,0.922334,0.922482,0.922093
Max ACF,0.864103,0.767004,0.859416,0.878973,0.877416,0.879489
Max PACF,0.906694,0.842115,0.898373,0.903542,0.908079,0.907030
None,0.899281,0.818601,0.894139,0.907084,0.908928,0.906779
PACF,0.917926,0.879048,0.910862,0.919491,0.917941,0.920480


#### Time: Subsequence Lenght Method 

In [ ]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,GunPoint,ItalyPowerDemand,SmoothSubspace,SonyAIBORobotSurface2
len_method,,,,,,,
ACF,0.073226,34.675459,142.606008,37.041264,0.353901,0.561044,0.700003
ACF&PACF,0.156653,117.498561,385.038644,71.945396,0.729001,0.930945,2.032898
Max ACF,0.076082,2.504580,51.043802,4.165960,0.361998,0.565765,0.548111
Max PACF,0.069733,6.387788,26.008793,4.032023,0.387409,0.639174,0.413045
None,0.054860,1.344463,23.982159,2.376824,0.331860,0.523768,0.305284
PACF,0.106152,93.916616,269.787351,47.107583,0.711325,0.843548,1.655537


In [ ]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,36.921862,1.897041,12.630965,51.699256,29.036795,35.597892
ACF&PACF,98.207529,4.024147,34.460765,137.398332,75.367775,93.642966
Max ACF,10.887561,0.696992,2.939326,15.207202,6.920173,9.288341
Max PACF,6.615012,0.656470,2.500908,8.702473,4.832249,5.910243
None,5.335883,0.607922,1.552838,7.181455,3.408100,4.402376
PACF,70.178082,2.926348,24.779742,97.838655,53.591008,66.384654


In [ ]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,36.107428,1.363858,7.659296,62.878754,22.483475,37.668420
ACF&PACF,99.807899,4.128269,22.601428,147.045945,64.983271,103.793068
Max ACF,9.452769,0.681231,2.603616,16.603925,6.555704,10.707254
Max PACF,6.561194,1.556164,2.652941,7.003523,4.970107,6.498364
None,4.887317,0.621602,1.547585,6.753864,3.541865,5.344212
PACF,73.306056,2.946412,16.732911,99.013718,47.457879,74.791964


#### Variance: Subsequence Lenght Method 

In [ ]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.009379,0.020168,0.011110,0.009207,0.009459,0.008864
ACF&PACF,0.005686,0.010392,0.008217,0.007457,0.007492,0.007824
Max ACF,0.008945,0.016718,0.011349,0.008405,0.009047,0.008636
Max PACF,0.005700,0.013935,0.006175,0.007870,0.006388,0.006915
None,0.006791,0.014641,0.007952,0.007920,0.007782,0.008007
PACF,0.006138,0.010470,0.008384,0.008831,0.008451,0.008506


In [ ]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.009243,0.019863,0.012487,0.009166,0.010822,0.010497
ACF&PACF,0.006475,0.010140,0.007707,0.007876,0.008121,0.007796
Max ACF,0.008432,0.018767,0.012596,0.009173,0.010339,0.009343
Max PACF,0.006182,0.013386,0.006408,0.007873,0.006828,0.008052
None,0.007353,0.014518,0.008476,0.009309,0.009154,0.008872
PACF,0.006785,0.010846,0.008357,0.008646,0.008123,0.008538


#### Another Results

In [ ]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,4770,4770,4770
unique,7,6,216
top,SonyAIBORobotSurface2,ACF&PACF,Max ACF: n_random_points= (lenthg ts)//2 nb_in...
freq,720,795,90


In [ ]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
Chinatown,0.017890
Coffee,6.408187
DistalPhalanxOutlineAgeGroup,26.205280
GunPoint,5.555635
ItalyPowerDemand,0.095850
SmoothSubspace,0.135475
SonyAIBORobotSurface2,0.188496


In [ ]:
df_result['time'].sum()/(60*60)

38.60681286752224

#### Generate CD Diagram Tunning Hyperparameter

In [ ]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

In [ ]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
Max ACF: n_random_points=1 nb_inst_per_class=1,214.785714
ACF: n_random_points=1 nb_inst_per_class=1,214.428571
Max PACF: n_random_points=1 nb_inst_per_class=1,208.357143
None: n_random_points=1 nb_inst_per_class=1,207.428571
ACF: n_random_points=10 nb_inst_per_class=1,207.214286
...,...
ACF&PACF: n_random_points=30 nb_inst_per_class=50,52.285714
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100,50.785714
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50,50.142857
ACF&PACF: n_random_points=30 nb_inst_per_class=100,50.000000


In [ ]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [ ]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=100 nb_inst_per_class=50'], dtype='object')

In [ ]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.920102,0.876336,0.907942,0.928765,0.929272,0.930861
1,0.906340,0.799072,0.899443,0.908865,0.915145,0.902210
10,0.916431,0.877807,0.914449,0.923763,0.921832,0.923350
100,0.927092,0.890890,0.911133,0.924247,0.924089,0.927289
30,0.925059,0.892965,0.909869,0.925987,0.920593,0.925977
50,0.919934,0.890108,0.913968,0.923297,0.924932,0.924125


In [ ]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.005996,0.007075,0.007797,0.006032,0.006560,0.006931
1,0.004583,0.018836,0.007260,0.006967,0.006415,0.008297
10,0.006572,0.007478,0.007407,0.007799,0.008623,0.008488
100,0.005459,0.007931,0.009878,0.008135,0.008255,0.008142
30,0.005494,0.009672,0.009579,0.008064,0.008816,0.007835
50,0.006079,0.006651,0.009211,0.009148,0.007905,0.008539


##### Overall time complexity

In [ ]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
Chinatown,0.000148,0.001009,0.000235,0.001189,0.002611
Coffee,0.001122,0.086429,0.000801,1.869862,1.958309
DistalPhalanxOutlineAgeGroup,0.000531,0.019499,0.016959,6.380078,6.417311
GunPoint,0.000645,0.027494,0.000965,1.169863,1.199090
ItalyPowerDemand,0.000136,0.001336,0.000842,0.009811,0.012150
SmoothSubspace,0.000103,0.002251,0.002864,0.010263,0.015516
SonyAIBORobotSurface2,0.000321,0.002770,0.000283,0.030471,0.033882


In [ ]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,1.961897,0.072595,0.699972,1.546370,1.868174,2.638054
1,0.080812,0.022441,0.069433,0.066345,0.074721,0.100790
10,0.437862,0.050059,0.171585,0.385562,0.487660,0.736154
30,1.305138,0.051585,0.486638,1.107244,1.418790,2.160659
50,2.057194,0.083651,0.773007,1.792802,2.272063,3.447349
100,2.677431,0.122874,1.263388,2.679917,3.286812,4.706554


##### Calculate weights time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.020784,0.023317,0.026846,0.018299,0.025531,0.021132
1,0.029550,0.025200,0.024992,0.026748,0.026968,0.027273
10,0.021205,0.025844,0.025044,0.022128,0.020663,0.024976
100,0.020387,0.025149,0.022801,0.027844,0.025168,0.025607
30,0.024191,0.026310,0.023317,0.026750,0.023481,0.024926
50,0.023690,0.026323,0.024516,0.025917,0.018355,0.025475


##### Finding subsequences time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.868319,0.091258,0.759585,1.320104,1.313920,1.359364
1,1.370147,0.380455,1.336793,1.570124,1.398944,1.433905
10,0.798511,0.093873,0.886825,1.439211,1.259811,1.402962
100,0.784165,0.101359,0.940220,1.751474,1.530082,1.751439
30,0.799063,0.089205,0.837139,1.564648,1.311978,1.436131
50,0.852628,0.132327,0.895811,1.761786,1.431132,1.468223


##### Transform Dataset time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,116.714159,4.062017,41.169162,156.812783,91.356209,110.606412
1,3.185973,0.612345,1.131704,4.426593,2.397978,2.995040
10,25.411805,1.134181,9.356679,42.654533,21.818717,27.786979
100,159.722923,7.133853,74.779738,280.398049,159.104486,195.266897
30,77.399114,2.856918,28.298623,127.924882,65.030026,83.582355
50,122.444249,4.729992,45.412443,204.888101,106.030705,134.719660


##### Classifier time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.100394,0.177987,0.038485,0.121695,0.078831,0.094542
1,0.257258,0.327041,1.670303,0.022017,0.154797,0.025255
10,0.038247,1.747896,0.024633,0.050754,0.032245,0.042656
100,0.112676,0.110268,0.057412,0.205926,0.128549,0.157464
30,0.081499,0.120931,0.035394,0.118315,0.065700,0.079556
50,0.104794,0.128173,0.044200,0.157411,0.082905,0.104694
